# SGP4 implementation

This notebook is a step-by-step implementation of the SGP4 algorithm in the heyoka.py expression system, adapted from the ``sgp4.f`` file here:

https://aim.hamptonu.edu/archive/cips/documentation/software/common/astron_lib/

The implementation incorporates also several code updates from the most recent version of the official C++ code from celestrak:

https://celestrak.org/software/vallado-sw.php

The code updates (flagged by comments in our implementation) include:

- workarounds for numerical issues for low eccentricity and/or inclination close to 180 degrees,
- an updated way to "un-Kozai" the mean motion.

The implementation splits the original code in blocks, so that it is possible to compare block-by-block the intermediate numerical values produced by this implementation with the intermediate values produced by the official C++ implementation.

On selected test cases, the agreement with the official C++ code is to machine epsilon ($\sim 10^{-16}$) up to the point of invoking the Kepler solver. The Kepler solver of the C++ implementation uses a tolerance of $10^{-12}$, while our Kepler solver tries to go down to machine precision. This introduces an "error" wrt the C++ implementation with a relative magnitude in the order of $\lesssim 10^{-13}$.

Comprehensive testing via the [sgp4 Python module](https://pypi.org/project/sgp4/) using the full catalog of LEO objects from [space track](http://www.space-track.org/) shows a maximum positional error of our implementation of around $10^{-5}\,\mathrm{m}$.

In [1]:
import heyoka as hy

# Small abs() wrapper.
def ABS(x):
    return hy.select(hy.gte(x, 0.), x, -x)

# ACTAN() wrapper.
def ACTAN(a, b):
    import math
    
    ret = hy.atan2(a, b)
    return hy.select(hy.gte(ret, 0.), ret, 2*math.pi + ret)

# max() wrapper.
def MAX(a, b):
    return hy.select(hy.gt(a, b), a, b)

# min() wrapper.
def MIN(a, b):
    return hy.select(hy.lt(a, b), a, b)

import numpy as np
np.set_printoptions(precision=16)

# Conversion from revolutions per day to
# rad per minute.
def revday2radmin(x):
    import math
    return x*2.*math.pi/1440.

In [2]:
# The inputs.
N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0 = hy.make_vars("N0", "I0", "E0", "BSTAR", "OMEGA0", "M0", "TSINCE", "NODE0")

In [3]:
# Numerical values for the inputs.
# NOTE: these are taken from the following TLE:
# 0 CBERS 1 DEB
# 1 32072U 99057QL  24163.54236549  .00515712  00000-0  11175-1 0  9995
# 2 32072  98.3151 237.3565 0011633 245.8809 114.1219 15.42369692   639
n0_val = revday2radmin(15.42369692)
i0_val = np.deg2rad(98.3151)
e0_val = .0011633
bstar_val = .11175e-1
omega0_val = np.deg2rad(245.8809)
m0_val = np.deg2rad(114.1219)
tsince_val = 34619.0103648098
node0_val = np.deg2rad(237.3565)

num_vals = [n0_val, i0_val, e0_val, bstar_val, omega0_val, m0_val, tsince_val, node0_val]

In [4]:
# Constants.
# NOTE: these are from the WGS72 model.
# It is possible to update these constants
# to WGS84 if needed, see the official C++
# code.
KE=0.07436691613317342
TOTHRD=2/3.
J2=1.082616e-3
CK2=.5*J2
KMPER=6378.135
S0=20./KMPER
S1=78./KMPER
Q0=120./KMPER
J3=-0.253881e-5
A3OVK2=-J3/CK2
J4=-0.00000165597
CK4=-.375*J4
SIMPHT=220./KMPER

In [5]:
# Recover original mean motion (N0DP) and semimajor axis (A0DP)
# from input elements.
A1 = (KE/N0)**TOTHRD
COSI0 = hy.cos(I0)
THETA2 = COSI0**2
X3THM1 = 3.*THETA2-1.
BETA02 = 1.-E0**2
BETA0 = hy.sqrt(BETA02)
DELA2 = 1.5*CK2*X3THM1/(BETA0*BETA02)
DEL1 = DELA2/A1**2
A0 = A1*(1.-DEL1*(1./3.+DEL1*(1.+134./81.*DEL1)))
DEL0 = DELA2/A0**2
N0DP = N0/(1.+DEL0)

In [6]:
cf = hy.cfunc([A1, COSI0, THETA2, X3THM1, BETA02, BETA0, DELA2, DEL1, A0, DEL0, N0DP], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf(num_vals)

res

array([ 1.0688479837924532e+00, -1.4461698027886044e-01,
        2.0914070984976309e-02, -9.3725778704507112e-01,
        9.9999864673311001e-01,  9.9999932336632613e-01,
       -7.6101925207740218e-04, -6.6613730990550167e-04,
        1.0690848431992177e+00, -6.6584217260072444e-04,
        6.7343413605752703e-02])

In [7]:
# Initialization for new element set.
#
# UPDATE: for the computation of A0DP,
# we use the new C++ code. The original
# Fortran code used this instead:
#
# A0DP = A0/(1.-DEL0)
#
# Not entirely sure how these two definitions
# relate - perhaps the original one is an
# approximated form of the updated one?
A0DP = (KE / N0DP )**TOTHRD
PERIGE = A0DP*(1.-E0)-1.
S = MIN(MAX(S0,PERIGE-S1),S1)
S4 = 1.+S
PINVSQ = 1./(A0DP*BETA02)**2
XI = 1./(A0DP-S4)
ETA = A0DP*XI*E0
ETASQ = ETA**2
EETA = E0*ETA
PSISQ = ABS(1.-ETASQ)
COEF = ((Q0-S)*XI)**4
COEF1 = COEF/(hy.sqrt(PSISQ)*PSISQ**3)
C1 = BSTAR*COEF1*N0DP*(A0DP*(1.+1.5*ETASQ+EETA*(4.+ETASQ))+0.75*CK2*XI/PSISQ*X3THM1*(8.+3.*ETASQ*(8.+ETASQ)))

In [8]:
cf = hy.cfunc([A0DP, PERIGE, S, S4, PINVSQ, XI, ETA, ETASQ, EETA, PSISQ, COEF, COEF1, C1], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf(num_vals)

res

array([1.0683734750822533e+00, 6.7130636218690171e-02,
       1.2229280189271628e-02, 1.0122292801892716e+00,
       8.7610262435118980e-01, 1.7811280434355368e+01,
       2.2136551533639422e-02, 4.9002691380147379e-04,
       2.5751450399082737e-05, 9.9950997308619849e-01,
       1.8923562487440558e-04, 1.8956053996213782e-04,
       1.4478799348179415e-07])

In [9]:
SINI0 = hy.sin(I0)
# UPDATE: clamp for small eccentricity values.
C3 = hy.select(hy.gt(E0, 1.0e-4), COEF*XI*A3OVK2*N0DP*SINI0/E0, 0.)
X1MTH2 = 1.-THETA2
C4 = 2.*N0DP*COEF1*A0DP*BETA02*(ETA*(2.+.5*ETASQ)+E0*(.5+2.*ETASQ)-2.*CK2*XI/(A0DP*PSISQ)*(-3.*X3THM1*(1.-2.*EETA+ETASQ*(1.5-.5*EETA))+.75*X1MTH2*(2.*ETASQ-EETA*(1.+ETASQ))*hy.cos(2.*OMEGA0)))
C5 = 2.*COEF1*A0DP*BETA02*(1.+2.75*(ETASQ+EETA)+EETA*ETASQ)
THETA4 = THETA2**2
TEMP1 = 3.*CK2*PINVSQ*N0DP
TEMP2 = TEMP1*CK2*PINVSQ
TEMP3 = 1.25*CK4*PINVSQ**2*N0DP
MDOT = N0DP+.5*TEMP1*BETA0*X3THM1+.0625*TEMP2*BETA0*(13.-78.*THETA2+137.*THETA4)
OMGDOT = -.5*TEMP1*(1.-5.*THETA2)+0.0625*TEMP2*(7.-114.*THETA2+395.*THETA4)+TEMP3*(3.-36.*THETA2+49.*THETA4)
HDOT1 = -TEMP1*COSI0
N0DOT = HDOT1+(.5*TEMP2*(4.-19.*THETA2)+2.*TEMP3*(3.-7.*THETA2))*COSI0
OMGCOF = BSTAR*C3*hy.cos(OMEGA0)
# UPDATE: clamp for small eccentricity values.
MCOF = hy.select(hy.gt(E0, 1.0e-4), -TOTHRD*COEF*BSTAR/EETA, 0.)
NODCF = 3.5*BETA02*HDOT1*C1
T2COF = 1.5*C1
# UPDATE: clamp for inclination close to 180 deg.
LCOF = .125*A3OVK2*SINI0*(3.+5.*COSI0)/hy.select(hy.gt(ABS(1.+COSI0), 1.5e-12),1.+COSI0,1.5e-12)
AYCOF = .25*A3OVK2*SINI0
DELM0 = (1.+ETA*hy.cos(M0))**3
SINM0 = hy.sin(M0)
X7THM1 = 7.*THETA2-1.

In [10]:
cf = hy.cfunc([SINI0, C3, X1MTH2, C4, C5, THETA4, TEMP1, TEMP2, TEMP3, MDOT, OMGDOT, HDOT1, N0DOT, OMGCOF, MCOF, NODCF, T2COF, LCOF, AYCOF, DELM0, SINM0, X7THM1], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf(num_vals)

res

array([ 9.8948771039110117e-01,  9.0551906424407916e-04,
        9.7908592901502367e-01, -1.6199118226530441e-07,
        4.0561687084828278e-04,  4.3739836516462789e-04,
        9.5811096041508684e-05,  4.5437584430632589e-08,
        4.0123502306567769e-08,  6.7298546243758917e-02,
       -4.2791427843667610e-05,  1.3855911386730865e-05,
        1.3810958581583456e-05, -4.1350466483793344e-06,
       -5.4746640809192580e-02,  7.0215841243219005e-12,
        2.1718199022269122e-07,  1.5441603063899533e-03,
        1.1602088339808536e-03,  9.7310453686073950e-01,
        9.1267803558518124e-01, -8.5360150310516580e-01])

In [11]:
# For perigee less than 220 kilometers, the equations are
# truncated to linear variation in sqrt A and quadratic
# variation in mean anomaly.  Also, the C3 term, the
# delta OMEGA term, and the delta M term are dropped.
C1SQ = C1**2
D2 = 4.*A0DP*XI*C1SQ
TEMP0 = D2*XI*C1/3.
D3 = (17.*A0DP+S4)*TEMP0
D4 = .5*TEMP0*A0DP*XI*(221.*A0DP+31.*S4)*C1
T3COF = D2+2.*C1SQ
T4COF = .25*(3.*D3+C1*(12.*D2+10.*C1SQ))
T5COF = .2*(3.*D4+12.*C1*D3+6.*D2**2+15.*C1SQ*(2.*D2+C1SQ))

In [12]:
cf = hy.cfunc([C1SQ, D2, TEMP0, D3, D4, T3COF, T4COF, T5COF], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf(num_vals)

res

array([2.0963563056484068e-14, 1.5956709152533625e-12,
       1.3716703957086080e-18, 2.6301201481926194e-17,
       5.0544922331187351e-22, 1.6375980413663305e-12,
       2.0426591262252964e-17, 3.1566639254383900e-22])

In [13]:
# Update for secular gravity and atmospheric drag.
MP = M0+MDOT*TSINCE
OMEGA = OMEGA0+OMGDOT*TSINCE
NODE = NODE0+(N0DOT+NODCF*TSINCE)*TSINCE
TEMPE = C4*TSINCE
TEMPA = 1.-C1*TSINCE
TEMPL = T2COF
TEMPF = MCOF*((1.+ETA*hy.cos(MP))**3-DELM0)+OMGCOF*TSINCE
# The conditional updates.
MP = MP + hy.select(hy.gte(PERIGE, SIMPHT), TEMPF, 0.)
OMEGA = OMEGA - hy.select(hy.gte(PERIGE, SIMPHT), TEMPF, 0.)
TEMPE = TEMPE + hy.select(hy.gte(PERIGE, SIMPHT), C5*(hy.sin(MP)-SINM0), 0.)
TEMPA = TEMPA - hy.select(hy.gte(PERIGE, SIMPHT),(D2+(D3+D4*TSINCE)*TSINCE)*TSINCE**2, 0.)
TEMPL = TEMPL + hy.select(hy.gte(PERIGE, SIMPHT),(T3COF+(T4COF+T5COF*TSINCE)*TSINCE)*TSINCE, 0.)
A = A0DP*TEMPA**2
N = KE/hy.sqrt(A**3)
E = E0-TEMPE*BSTAR
# UPDATE: clamp for small eccentricity values.
E = hy.select(hy.lt(E, 1e-6), 1e-6, E)
TEMPL = TEMPL*TSINCE**2

In [14]:
cf = hy.cfunc([MP, OMEGA, NODE, TEMPE, TEMPA, TEMPL, A, N, E], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf(num_vals)

res

array([ 2.3316535179932898e+03,  2.9573892483855082e+00,
        4.6291893434965505e+00, -5.7519067229317112e-03,
        9.9125797253869452e-01,  3.7326747535417422e+02,
        1.0497756229192925e+00,  6.9140902940034193e-02,
        1.2275775576287618e-03])

In [15]:
# Long period periodics.
AXN = E*hy.cos(OMEGA)
AB = A*(1.-E**2)
AYN = AYCOF/AB+E*hy.sin(OMEGA)

In [16]:
cf = hy.cfunc([AXN, AB, AYN], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf(num_vals)

res

array([-0.0012068099769773,  1.0497740409634237,  0.0013300460012562])

In [17]:
# NOTE: this is the original Kepler-like solver, commented out
# because we can use the kepF() function from heyoka.py in its place.
# CAPU = FMOD2P(LCOF*AXN/AB+MP+OMEGA+N0DP*TEMPL)
# EPWNEW = CAPU
# for _ in range(10):
#     EPW = EPWNEW
#     SINEPW = hy.sin(EPW)
#     COSEPW = hy.cos(EPW)
#     ESINE = AXN*SINEPW-AYN*COSEPW
#     ECOSE = AXN*COSEPW+AYN*SINEPW
#     EPWNEW = (CAPU+ESINE-EPW)/(1.-ECOSE)+EPW

In [18]:
# Solve Kepler's equation.
CAPU = LCOF*AXN/AB+MP+OMEGA+N0DP*TEMPL
EPWNEW = hy.kepF(AYN, AXN, CAPU)
SINEPW = hy.sin(EPWNEW)
COSEPW = hy.cos(EPWNEW)
ESINE = AXN*SINEPW-AYN*COSEPW
ECOSE = AXN*COSEPW+AYN*SINEPW

In [19]:
cf = hy.cfunc([CAPU, SINEPW, COSEPW, ESINE, ECOSE], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf(num_vals)

res

array([ 2.3597480114448749e+03, -4.0193725348674864e-01,
       -9.1566721261576756e-01,  1.7029414022476496e-03,
        5.7044129101975450e-04])

In [20]:
# Short period preliminary quantities
ELSQ = AXN**2+AYN**2
TEMPS = 1.-ELSQ
PL = A*TEMPS
R = A*(1.-ECOSE)
RDOT = KE*hy.sqrt(A)*ESINE/R
RFDOT = KE*hy.sqrt(PL)/R
BETAL = hy.sqrt(TEMPS)
TEMP3 = ESINE/(1.+BETAL)
COSU = (COSEPW-AXN+AYN*TEMP3)*A/R
SINU = (SINEPW-AYN-AXN*TEMP3)*A/R
U = ACTAN(SINU,COSU)
SIN2U = 2.*SINU*COSU
COS2U = 2.*COSU**2-1.
TEMP1 = CK2/PL
TEMP2 = TEMP1/PL

In [21]:
cf = hy.cfunc([ELSQ, TEMPS, PL, R, RDOT, RFDOT, BETAL, TEMP3, COSU, SINU, U, SIN2U, COS2U, TEMP1, TEMP2], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf(num_vals)

res

array([ 3.2254126859894749e-06,  9.9999677458731406e-01,
        1.0497722369596809e+00,  1.0491767875576734e+00,
        1.2367418156586886e-04,  7.2623744981675939e-02,
        9.9999838729235657e-01,  8.5147138771103235e-04,
       -9.1498121320719650e-01, -4.0349644295568049e-01,
        3.5569276221491659e+00,  7.3838332980075383e-01,
        6.7438124104422625e-01,  5.1564328045835932e-04,
        4.9119538725061931e-04])

In [22]:
# Update for short periodics.
RK = R*(1.-1.5*TEMP2*BETAL*X3THM1)+.5*TEMP1*X1MTH2*COS2U
UK = U-.25*TEMP2*X7THM1*SIN2U
NODEK = NODE+1.5*TEMP2*COSI0*SIN2U
IK = I0+1.5*TEMP2*COSI0*SINI0*COS2U
RDOTK = RDOT-N*TEMP1*X1MTH2*SIN2U
RFDOTK = RFDOT+N*TEMP1*(X1MTH2*COS2U+1.5*X3THM1)

In [23]:
cf = hy.cfunc([RK, UK, NODEK, IK, RDOTK, RFDOTK], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf(num_vals)

res

array([1.0500715449591582e+00, 3.5570050204350880e+00,
       4.6291106666923412e+00, 1.7158510976081480e+00,
       9.7899868348241692e-05, 7.2597162480580721e-02])

In [24]:
# Orientation vectors.
SINUK = hy.sin(UK)
COSUK = hy.cos(UK)
SINIK = hy.sin(IK)
COSIK = hy.cos(IK)
SINNOK = hy.sin(NODEK)
COSNOK = hy.cos(NODEK)
MX = -SINNOK*COSIK
MY = COSNOK*COSIK
UX = MX*SINUK+COSNOK*COSUK
UY = MY*SINUK+SINNOK*COSUK
UZ = SINIK*SINUK
VX = MX*COSUK-COSNOK*SINUK
VY = MY*COSUK-SINNOK*SINUK
VZ = SINIK*COSUK

In [25]:
cf = hy.cfunc([SINUK, COSUK, SINIK, COSIK, SINNOK, COSNOK, MX, MY, UX, UY, UZ, VX, VY, VZ], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf(num_vals)

res

array([-0.4035672597245912, -0.9149499805335722,  0.9894979904203342,
       -0.1445466255369538, -0.9965343648574752, -0.0831820873632574,
       -0.1440456796717595,  0.0120236900334789,  0.1342395694340525,
        0.9069267300887937, -0.3993289924969241,  0.0982250247562603,
       -0.4131697178089905, -0.9053411670730935])

In [26]:
# Position and Velocity
PV1 = RK*UX
PV2 = RK*UY
PV3 = RK*UZ
PV4 = RDOTK*UX+RFDOTK*VX
PV5 = RDOTK*UY+RFDOTK*VY
PV6 = RDOTK*UZ+RFDOTK*VZ

In [27]:
sgp4 = hy.cfunc([PV1, PV2, PV3, PV4, PV5, PV6], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0], fast_math=True)

res = sgp4(num_vals)

res

array([ 0.1409611520702677,  0.9523379526290973, -0.4193240120982288,
        0.007144000118064 , -0.0299061611283577, -0.065764294062157 ])

In [28]:
print(res[:3] * KMPER)
print(res[3:] * KMPER/60.)

[  899.0692576596966  6074.140027491988  -2674.5051579041365]
[ 0.7594232865504702 -3.179092216806967  -6.990892428468935 ]
